### Lab: Logistic Regression

**Task:** In order to optimize our inventory, we would like to know which films will be 
      rented next month and we are asked to create a model to predict it.

### 1- Import libraries

In [ ]:
import pymysql
from sqlalchemy import create_engine
import pandas as pd
import numpy as np
import getpass
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix
password = getpass.getpass()

## check with script how to import data

### 2- Import data and load into dataframe

Create an SQL query or queries to extract the information you think may be relevant for building the prediction model. It should include some film features and some rental features

### 3- Prepare data for the model

Analyze extracted features and transform them. You may need to encode some categorical variables, or scale numerical variables*

**Explore Data**

In [ ]:
data.shape

In [ ]:
data.dtypes

In [ ]:
data.describe()

In [ ]:
data.isna().sum()

**Clean Data**

In [ ]:
Drop Nan values

In [ ]:
change column types

**EDA**

In [ ]:
correlation matrix

In [ ]:
distribution of numerical columns

In [ ]:
encode categorical data

### 4- Work on Target Variable

Create a query to get the list of films and a boolean indicating if it was rented last month. 
This would be our target variable.

### 5- Create a logistic regression model to predict this variable from the cleaned data

### 6- Evaluate the results (calculate the score of the model)